# deep-audiobook-tuner
A system that generates an apt, emotionally pertinent, unique musical score for an audiobook automatically based on the current narrative for the purpose of ameliorating user-experience while being accurate, cost-efficient, and time saving.

### Dependencies

In [ ]:
import os
import sys
sys.path.append(os.path.abspath("../"))
import tensorflow as tf
from deepaudiobooktuner.deep_audiobook_tuner import *

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Create an instance of the deepAudiobookTuner Class

In [3]:
dat = deepAudiobookTuner()

### Initializing the instance withe audiobook path

In [4]:
dat.initialize(audiobook_path = r'D:\Projects\BEProject\deep-audiobook-tuner\assets\audiobooks\Alice_in_Wonderland_chap_1_mp3cut.net.mp3')


Creating temporary directory.
----Temporary directory created.

Loading assets.


C:\Users\Danny\miniconda3\envs\deepaudiobooktuner\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\Danny\miniconda3\envs\deepaudiobooktuner\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


----Loaded audio model assets. Time taken: 6.0 s
----Loaded text model assets. Time taken: 20.3 s
----Loaded music model assets. Time taken: 7.6 s
----Setup IBM transcription service. Time taken: 0.0 s

Converting mp3 to wav
----Converted to wav. Time taken: 0.7 s

Segmenting audiobook
----Segmented audio file. Time taken: 12.5 s


### Performing Sentiment analysis by invoking the analyzeSentiments() function

In [5]:
dat.analyzeSentiments()



Performing sentiment analysis

Processing clip 1:
----Text sentiment analysis
--------Transcription complete. Time taken: 20.2 s
--------Text analysis complete. Time taken: 17.5 s
----Audio sentiment analysis
--------Audio analysis complete. Time taken: 3.1 s
----Predicting final emotion
----Clip 1 processed. Time taken: 40.8 s

Processing clip 2:
----Text sentiment analysis
--------Transcription complete. Time taken: 17.6 s
--------Text analysis complete. Time taken: 0.7 s
----Audio sentiment analysis
--------Audio analysis complete. Time taken: 4.1 s
----Predicting final emotion
----Clip 2 processed. Time taken: 22.4 s

Processing clip 3:
----Text sentiment analysis
--------Transcription complete. Time taken: 17.5 s
--------Text analysis complete. Time taken: 0.7 s
----Audio sentiment analysis
--------Audio analysis complete. Time taken: 3.9 s
----Predicting final emotion
----Clip 3 processed. Time taken: 22.1 s

Processing clip 4:
----Text sentiment analysis
--------Transcription 

### Generating Music by invoking the generateMusic() function

In [6]:
dat.generateMusic(music_emotions=["Angry", "Happy", "Neutral", "Sad"])



Generating music


----generated Angry clip. Time taken: 45.0 s


----generated Happy clip. Time taken: 44.5 s


----generated Neutral clip. Time taken: 35.7 s


----generated Sad clip. Time taken: 33.1 s
----
Music Generation Complete. Time taken: 167.3 s


### Generating the final soundtrack and overlaying it on the audiobook

In [8]:
dat.generateSoundtrack()

### Final audiobook generated by DAT

In [9]:
dat.final_audiobook

### Deleting the temperory directory once the user is done

In [ ]:
dat.deleteTempDirectory()